In [1]:
import pandas as pd
import json
from pathlib import Path
from pprint import pprint

In [12]:
df_py=pd.read_csv("../datasets/python-code.csv")
ds_path = Path("../datasets/ds-code.jsonl")

In [10]:
df_py.head()
df_py.isna().sum()
df_py.duplicated().sum()

np.int64(0)

In [13]:
with ds_path.open("r", encoding="utf-8") as f:
    ds_data=[json.loads(line) for line in f]

print("total records: ", len(ds_data))
pprint(ds_data[0], depth=2)

total records:  17311
{'code_len': 745,
 'dataset': 'code-instructions-122k-alpaca-style',
 'has_comp': False,
 'has_ml': True,
 'has_nn': True,
 'has_plt': False,
 'has_wrang': False,
 'input': '',
 'instruction': 'Design a neural network model to classify emails into spam '
                'and non-spam.',
 'instruction_len': 72,
 'lang': 'python',
 'mean_ln_len': 30.0833333333,
 'nl_ratio': 0.1651006711,
 'output': 'import tensorflow as tf\n'
           'from tensorflow.keras.layers import Dense, Input\n'
           '\n'
           '# Preparing data\n'
           'X_train, y_train, X_test, y_test = get_data() # Returns training '
           'and testing datasets\n'
           '\n'
           '# Building model\n'
           'inputs = Input(shape=(X_train.shape[1], ))\n'
           "x = Dense(64, activation='relu')(inputs)\n"
           "x = Dense(32, activation='relu')(x)\n"
           "predictions = Dense(1, activation='sigmoid')(x)\n"
           '\n'
           '# Compiling model\n

In [14]:
df_py["Input"]=df_py["Input"].fillna("")
df_py=df_py.rename(columns={
    "Input": "input",
    "Output": "output"
})

In [16]:
python_data=[]
for _, row in df_py.iterrows():
    entry={
        "prompt": f"### Instruction:\n{row['Instruction']}\n\n### Input:\n{row['input']}\n\n### Response:\n",
        "response": row['output']
    }
    python_data.append(entry)

In [18]:
Path("../processed-data").mkdir(exist_ok=True)
with open("../processed-data/python_instruct.jsonl", "w", encoding="utf-8") as f:
    for item in python_data:
        f.write(json.dumps(item, ensure_ascii=False)+"\n")

In [20]:
def clean_text(text):
    return (
        text.replace("\u2028", " ").replace("\u2029", " ").replace("\r", "").strip()
    )

In [21]:
ds_data=[]
with open("../datasets/ds-code.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        data = json.loads(line)
        ds_data.append(data)

ds_clean=[]
for item in ds_data:
    instruction=clean_text(item.get("instruction", "").strip())
    input_=clean_text(item.get("input", "").strip())
    output=clean_text(item.get("output", "").strip())

    formatted={
        "prompt": f"### Instruction:\n{instruction}\n\n### Input:\n{input_}\n\n### Response:\n",
        "response": output
    }
    ds_clean.append(formatted)

with open("../processed-data/ds_coder_instruct.jsonl", "w", encoding="utf-8") as f:
    for item in ds_clean:
        f.write(json.dumps(item, ensure_ascii=False)+"\n")